>
> # MaaSSim tutorial

 ### spatial patterns

In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
!pip install h3

     |████████████████████████████████| 686 kB 130 kB/s            


In [14]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import Image
import selenium.webdriver
import shapely.geometry
import osmnx as ox
import pyproj
from geojson.feature import Feature, FeatureCollection
from shapely.geometry import Point, Polygon, LineString
phantom_path = '/Users/rafalkucharski/anaconda3/lib/node_modules/phantomjs-prebuilt/lib/phantom/bin/phantomjs'
import h3
import json

In [15]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
import seaborn as sns
import h3
import pandas as pd
import matplotlib.pyplot as plt
from dotmap import DotMap
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

prepare

In [16]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, collect_results  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate, simulate_parallel
from MaaSSim.traveller import travellerEvent
from MaaSSim.driver import driverEvent

In [17]:
params = get_config('../../data/config/delft.json')  # load configuration

In [18]:
params.paths.dumps = 'hexes'
params.times.patience = 1200
params.simTime = 4
params.parallel.nThread = 4
params.parallel.nReplications = 20

In [19]:
inData = load_G(inData, params)  # load network graph 

In [20]:
space = DotMap()
space.nP = [200]
space.nV = [10]
dict(space)

{'nP': [200], 'nV': [10]}

simulate

In [21]:
simulate_parallel(params = params, search_space = space)

08-02-22 19:20:20-WARNING-Setting up 4h simulation at 2022-02-08 17:20:23 for 10 vehicles and 200 passengers in Delft, Netherlands
08-02-22 19:20:20-WARNING-Setting up 4h simulation at 2022-02-08 17:21:44 for 10 vehicles and 200 passengers in Delft, Netherlands
08-02-22 19:20:21-WARNING-Setting up 4h simulation at 2022-02-08 17:20:25 for 10 vehicles and 200 passengers in Delft, Netherlands
08-02-22 19:20:21-WARNING-Setting up 4h simulation at 2022-02-08 17:20:14 for 10 vehicles and 200 passengers in Delft, Netherlands
08-02-22 19:20:26-WARNING-simulation time 4.9 s
08-02-22 19:20:26-WARNING-simulation time 4.8 s
08-02-22 19:20:26-WARNING-assertion tests for simulation results - passed
08-02-22 19:20:26-WARNING-simulation time 4.9 s
08-02-22 19:20:26-WARNING-assertion tests for simulation results - passed
08-02-22 19:20:26-WARNING-simulation time 4.8 s
08-02-22 19:20:26-WARNING-assertion tests for simulation results - passed
08-02-22 19:20:26-WARNING-assertion tests for simulation resul

### collect

demand structure

In [10]:
res = collect_results('hexes')

In [35]:
def plot_hex_wait_vehs(inData, res, APERTURE_SIZE = 9, threshold = 3, col = 'wait_time'):
       
    traces = res.rides[['veh','pos','t']]
    diffs = traces.diff()
    diffs.columns = [_+"_diff" for _ in diffs.columns]
    traces = pd.concat([traces, diffs], axis = 1)
    trips = traces[(traces.veh_diff==0) & (traces.pos_diff==0)][['pos','t_diff']]
    trips.columns = ['pos','wait_time']
    
    trips['x'] = trips.apply(lambda row: inData.nodes.loc[row.pos].x, axis = 1)
    trips['y'] = trips.apply(lambda row: inData.nodes.loc[row.pos].y, axis = 1)
    trips['hex_o_{}'.format(APERTURE_SIZE)] = trips.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    
    
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 30]
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
plot_hex_wait_vehs(inData, res)

In [29]:
res = collect_results("hexes")
def plot_hex_map(inData, res, APERTURE_SIZE = 9, threshold = 3, col = 'wait_time'):
    res.requests['wait_time'] = res.requests.apply(lambda x: res.pax_exp[res.pax_exp['dt'] == x['dt']].loc[x.name].MEETS_DRIVER_AT_PICKUP, axis = 1)
    res.requests['x'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].x, axis = 1)
    res.requests['y'] = res.requests.apply(lambda row: inData.nodes.loc[row.origin].y, axis = 1)
    res.requests['hex_o_{}'.format(APERTURE_SIZE)] = res.requests.apply(lambda row: h3.geo_to_h3(row.y,row.x,APERTURE_SIZE),axis = 1) 
    trips = res.requests
    
    col_geom = 'hex_o_{}'.format(APERTURE_SIZE)
    hexes = pd.Series(list(set(list(trips[col_geom].unique())+list(trips[col_geom].unique())))).to_frame(col_geom)
    hexes = hexes.set_index(col_geom)
    hexes[col_geom] = hexes.index.copy()
    hexes['nobs'] = trips.groupby(col_geom).size()
    hexes = hexes[hexes['nobs']>threshold]
    hexes['geom'] = hexes.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[col_geom], geo_json = True)]}, axis = 1)
    aggr = trips.groupby(col_geom)[col]
    hexes['nobs'] = aggr.size()
    hexes[col] = aggr.mean()/60
    hexes[col+'_std'] = aggr.std()
    list_features = []
    for i, row in hexes.iterrows():
        feature = Feature(geometry = row["geom"],
                          id = row[col_geom],
                          properties = {"resolution": 9})
        list_features.append(feature)

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    CENTER = list(inData.nodes.loc[inData.stats.center][['y','x']].values)
    tile = 'cartodbpositron'
    base_map = folium.Map(location=CENTER, zoom_start=13,tiles=tile, zoomControl =  False)
    bins = [0, 1, 2, 3, 5, 10, 15, 20, 25, 114]
    m = folium.Choropleth(geo_data = geojson_hexes,data = hexes, columns = [col_geom,col],key_on ="feature.id",
            fill_color='Blues', control = False, bins = bins,
            fill_opacity=0.7, line_opacity=0.1).add_to(base_map)
    return base_map
    
plot_hex_map(inData, res, APERTURE_SIZE = 9)

-------------
Rafal Kucharski, 2020, Delft, r.m.kucharski@tudelft.nl